## Chapter 3: The Service Defaults Project – A Deep Dive into Production Readiness

In Chapter 2, we explored the Aspire starter template and saw how the **ServiceDefaults** project provides a foundation of observability, resilience, health checks, and service discovery with just one line of code: `builder.AddServiceDefaults()`. But what really happens inside that method? How can you customize it to meet your specific requirements? In this chapter, we’ll peel back the layers and examine each pillar of the ServiceDefaults project in depth. By the end, you’ll understand not only how to use the defaults but also how to extend and tailor them for production‑grade applications.

---

### 3.1 Recap: What We’ve Seen So Far

The ServiceDefaults project is a class library referenced by every service in your Aspire solution. Its `Extensions.cs` file contains an extension method `AddServiceDefaults` that configures:

- **OpenTelemetry** for logs, metrics, and traces.
- **Health checks** (a basic “self” check).
- **Service discovery** (so HTTP clients can resolve logical names).
- **Resilience** (default retry, timeout, and circuit‑breaker policies for HTTP clients).

In this chapter, we’ll go beyond the basics and understand the internals of each of these features. We’ll also see how to customize them for real‑world scenarios.

---

### 3.2 The ServiceDefaults Project Structure

Let’s open the `ServiceDefaults` project in our `MyAspireApp` solution. You’ll see a single file: `Extensions.cs`. This file contains several private methods that are called by `AddServiceDefaults`. Here’s an annotated version of the key methods:

```csharp
// ServiceDefaults/Extensions.cs
using Microsoft.AspNetCore.Builder;
using Microsoft.AspNetCore.Diagnostics.HealthChecks;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Diagnostics.HealthChecks;
using Microsoft.Extensions.Logging;
using OpenTelemetry;
using OpenTelemetry.Metrics;
using OpenTelemetry.Trace;

namespace Microsoft.Extensions.Hosting;

public static class Extensions
{
    public static IHostApplicationBuilder AddServiceDefaults(this IHostApplicationBuilder builder)
    {
        builder.ConfigureOpenTelemetry();
        builder.AddDefaultHealthChecks();
        builder.Services.AddServiceDiscovery();
        builder.Services.ConfigureHttpClientDefaults(http =>
        {
            http.AddStandardResilienceHandler();
            http.AddServiceDiscovery();
        });
        return builder;
    }

    // ... private helper methods
}
```

The `AddServiceDefaults` method orchestrates the setup. Let’s examine each part in detail.

---

### 3.3 OpenTelemetry Deep Dive

OpenTelemetry (OTel) is the industry standard for observability in cloud‑native applications. It provides three pillars:

- **Traces**: Track the path of a request as it travels through services.
- **Metrics**: Measure system performance (request rates, error counts, latency).
- **Logs**: Record discrete events with structured data.

Aspire’s `ConfigureOpenTelemetry` method sets up all three with sensible defaults and exports them to the Aspire Dashboard via the OTLP protocol.

#### 3.3.1 How Aspire Configures OpenTelemetry

Look at the `ConfigureOpenTelemetry` method inside `Extensions.cs`:

```csharp
public static IHostApplicationBuilder ConfigureOpenTelemetry(this IHostApplicationBuilder builder)
{
    builder.Logging.AddOpenTelemetry(logging =>
    {
        logging.IncludeFormattedMessage = true;
        logging.IncludeScopes = true;
    });

    builder.Services.AddOpenTelemetry()
        .WithMetrics(metrics =>
        {
            metrics.AddAspNetCoreInstrumentation()
                   .AddHttpClientInstrumentation()
                   .AddRuntimeInstrumentation();
        })
        .WithTracing(tracing =>
        {
            tracing.AddAspNetCoreInstrumentation()
                   .AddHttpClientInstrumentation()
                   .AddEntityFrameworkCoreInstrumentation();
        });

    builder.Services.AddOpenTelemetryExporters();

    return builder;
}
```

**Breakdown**:

- **Logging**: `builder.Logging.AddOpenTelemetry` hooks into the .NET logging pipeline. It sends all log messages (with scopes and formatted messages) to OpenTelemetry, which then forwards them to the configured exporter.
- **Metrics**: `WithMetrics` adds instrumentations that collect:
  - `AddAspNetCoreInstrumentation`: HTTP server metrics (request count, duration, etc.).
  - `AddHttpClientInstrumentation`: HTTP client metrics (outgoing request metrics).
  - `AddRuntimeInstrumentation`: .NET runtime metrics (GC, thread pool, CPU, memory).
- **Tracing**: `WithTracing` adds similar instrumentations for distributed tracing, capturing spans for incoming HTTP requests, outgoing HTTP calls, and Entity Framework Core database operations.
- **Exporters**: `AddOpenTelemetryExporters` checks for the `OTEL_EXPORTER_OTLP_ENDPOINT` environment variable (set by the AppHost) and configures the OTLP exporter to send all telemetry to that endpoint.

#### 3.3.2 Understanding Traces, Spans, and Attributes

When a request hits your API, OpenTelemetry creates a **trace**—a tree of **spans**. Each span represents a unit of work (e.g., an HTTP request, a database query). Spans contain:

- **Name**: e.g., “HTTP GET /weatherforecast”.
- **Start and end timestamps**.
- **Attributes**: key‑value pairs (e.g., HTTP method, status code, user ID).
- **Events**: timestamped annotations (e.g., “cache hit”).
- **Status**: OK or error.

The Aspire Dashboard visualizes traces as Gantt charts, showing you exactly where time is spent.

#### 3.3.3 Customizing Telemetry

You may need to add your own custom metrics or enrich traces with business‑specific attributes. For example, suppose you want to count the number of orders processed. You can add a custom metric:

1. In your service’s `Program.cs`, after `AddServiceDefaults`, access the `Meter` API:

```csharp
using System.Diagnostics.Metrics;

var builder = WebApplication.CreateBuilder(args);
builder.AddServiceDefaults();

var meter = new Meter("MyApp.Orders", "1.0.0");
var orderCounter = meter.CreateCounter<int>("orders.processed", description: "Number of orders processed");

// ... later in your endpoint
app.MapPost("/orders", (Order order) =>
{
    // process order...
    orderCounter.Add(1);
    return Results.Ok();
});
```

2. To see this metric in the dashboard, you must ensure the meter is registered with OpenTelemetry. Modify the metrics configuration in your service (not in ServiceDefaults, but in the service’s own code) to include your meter:

```csharp
builder.Services.AddOpenTelemetry()
    .WithMetrics(metrics => metrics
        .AddMeter("MyApp.Orders")   // Add your custom meter
        .AddAspNetCoreInstrumentation()
        .AddHttpClientInstrumentation()
        .AddRuntimeInstrumentation());
```

But wait—`AddServiceDefaults` already configures metrics. How can we add our own meter without duplicating code? You can chain the configuration after calling `AddServiceDefaults` because `AddServiceDefaults` returns the builder. However, the default `AddServiceDefaults` doesn’t expose the metrics builder for further customization. A better pattern is to modify the ServiceDefaults project to accept additional configuration. We’ll cover that in the hands‑on section.

---

### 3.4 Health Checks

Health checks are essential for orchestration platforms (like Kubernetes) and for the Aspire Dashboard to report the status of your services.

#### 3.4.1 The ASP.NET Core Health Checks Framework

ASP.NET Core provides a built‑in health checks system. You register one or more `IHealthCheck` implementations, and then expose them via middleware at a specific endpoint (e.g., `/health`). The endpoint returns a summary status (Healthy, Degraded, Unhealthy) and details of each check.

In ServiceDefaults, `AddDefaultHealthChecks` does:

```csharp
public static IHostApplicationBuilder AddDefaultHealthChecks(this IHostApplicationBuilder builder)
{
    builder.Services.AddHealthChecks()
        .AddCheck("self", () => HealthCheckResult.Healthy(), ["live"]);
    return builder;
}
```

This adds a single check named “self” that always returns Healthy. The `["live"]` tag marks it as a liveness check (in Kubernetes, liveness probes determine when to restart a container). Typically you also have **readiness** checks that indicate whether the service is ready to accept traffic (e.g., database is available).

#### 3.4.2 How Aspire Exposes Health Endpoints

Aspire does not automatically map the health endpoint; you must do it yourself in each service’s `Program.cs`. The starter template’s `ApiService` and `Web` projects do not explicitly map `/health`, but the dashboard still shows health status. Why? Because Aspire’s service defaults include a **health check service** but not the endpoint mapping. However, the dashboard probes the standard health endpoint (`/health`) if it exists. In the template, it doesn’t exist, so the dashboard shows “Unknown”. To enable it, you need to add:

```csharp
app.MapHealthChecks("/health");
```

But wait—the template’s `Program.cs` doesn’t have that line. Actually, if you look closely, the `AddServiceDefaults` method does **not** map the endpoint. So how does the dashboard get health status? In the starter template, the dashboard actually shows health as “Healthy” based on the resource being running, not from a health check endpoint. To get true health check integration, you must map the endpoint. Let’s fix that.

In `ApiService/Program.cs`, after `app.Build()` but before `app.Run()`, add:

```csharp
app.MapHealthChecks("/health");
```

Now, when you run the app, the dashboard will query `/health` and display the result.

#### 3.4.3 Creating Advanced Health Checks

Health checks can verify connectivity to databases, external APIs, or any dependency. Let’s create a readiness check that verifies the database connection (we’ll add a database in Chapter 4). For now, we’ll simulate.

Create a new class `DatabaseHealthCheck` in the `ApiService` project:

```csharp
using Microsoft.Extensions.Diagnostics.HealthChecks;

namespace MyAspireApp.ApiService.HealthChecks;

public class DatabaseHealthCheck : IHealthCheck
{
    public Task<HealthCheckResult> CheckHealthAsync(
        HealthCheckContext context,
        CancellationToken cancellationToken = default)
    {
        // Simulate a database check
        bool databaseIsAvailable = true; // In reality, try to open a connection

        if (databaseIsAvailable)
        {
            return Task.FromResult(HealthCheckResult.Healthy("Database is reachable"));
        }
        return Task.FromResult(HealthCheckResult.Unhealthy("Database is unreachable"));
    }
}
```

Register it in `Program.cs`:

```csharp
builder.Services.AddHealthChecks()
    .AddCheck<DatabaseHealthCheck>("database");
```

Now your service has two health checks: “self” (from defaults) and “database” (custom). When you map `/health`, the endpoint will aggregate them.

#### 3.4.4 Liveness vs. Readiness

In production (e.g., Kubernetes), you typically have two separate health endpoints:

- **Liveness**: indicates if the application is alive (if it fails, the pod is restarted).
- **Readiness**: indicates if the application is ready to accept traffic (if it fails, traffic is stopped).

You can tag your health checks accordingly. For example:

```csharp
.AddCheck<DatabaseHealthCheck>("database", tags: ["ready"]);
```

Then map two endpoints:

```csharp
app.MapHealthChecks("/health/live", new HealthCheckOptions { Predicate = check => check.Tags.Contains("live") });
app.MapHealthChecks("/health/ready", new HealthCheckOptions { Predicate = check => check.Tags.Contains("ready") });
```

The “self” check is tagged `["live"]`, so it would appear in the liveness probe. The database check could be tagged `["ready"]` so that if the database is down, the service is marked not ready, but it won’t be restarted (it might recover when the database comes back).

---

### 3.5 Resilience with Polly

The line `http.AddStandardResilienceHandler()` adds a Polly pipeline to every `HttpClient` registered in the application. This pipeline includes:

- **Retry**: Retries failed requests with exponential backoff. Default: up to 3 retries for HTTP 5XX, timeout, or network errors.
- **Circuit breaker**: Breaks the circuit (stops sending requests) after a certain number of failures, giving the downstream service time to recover.
- **Timeout**: Ensures requests don’t hang indefinitely (default 30 seconds).
- **Rate limiting**: (Sometimes included) limits the number of requests.

These policies are essential for building resilient microservices. Without them, a transient failure (like a brief network glitch) could cause a cascading failure.

#### 3.5.1 Inspecting the Default Policies

The `AddStandardResilienceHandler` is an extension from the `Microsoft.Extensions.Http.Resilience` package. You can customize the policies per `HttpClient` by calling `AddResilienceHandler` and providing your own pipeline. For example, to change the retry count:

```csharp
builder.Services.AddHttpClient<WeatherApiClient>(client =>
{
    client.BaseAddress = new Uri("http://apiservice");
})
.AddResilienceHandler("my-pipeline", builder =>
{
    builder.AddRetry(new HttpRetryStrategyOptions
    {
        MaxRetryAttempts = 5,
        Delay = TimeSpan.FromSeconds(2),
        BackoffType = DelayBackoffType.Exponential
    });
    // also add timeout, circuit breaker, etc.
});
```

But the beauty of `AddStandardResilienceHandler` is that it applies to all clients automatically. If you need different policies for different clients, you can override.

#### 3.5.2 Testing Resilience Locally

You can test resilience by deliberately causing failures. For example, stop the API service while the web frontend is running, and watch the web frontend retry. You’ll see logs in the dashboard showing retry attempts. This is a great way to validate your resilience strategy.

---

### 3.6 Service Discovery in Depth

Service discovery allows your services to find each other using logical names instead of hard‑coded URLs. In Aspire, the AppHost injects environment variables containing the actual endpoints. The `AddServiceDiscovery` method registers a resolver that reads these variables.

#### 3.6.1 How It Works

When you add a project resource with a logical name, e.g., `"apiservice"`, the AppHost sets environment variables like:

- `services__apiservice__http__0` = `http://localhost:5001`
- `services__apiservice__https__0` = `https://localhost:5002`

The number suffix (`0`) allows for multiple endpoints (e.g., if the service listens on multiple ports). The service discovery resolver reads these variables and maps the logical name `"apiservice"` to the appropriate URI scheme.

When you create an `HttpClient` with `BaseAddress = new Uri("http://apiservice")`, the `ServiceDiscoveryHandler` intercepts the request and replaces `"apiservice"` with the actual resolved address. This all happens transparently.

#### 3.6.2 Customizing Service Discovery

You might need to integrate with a different service discovery system (e.g., Consul, Kubernetes DNS). The `AddServiceDiscovery` method accepts a delegate to configure options. You can also implement your own `IServiceEndpointProvider`. However, for most Aspire applications, the built‑in resolver is sufficient.

#### 3.6.3 Using Service Discovery with gRPC or Other Protocols

Service discovery works with any URI scheme. If you have a gRPC service, you can use `https://apiservice` and Aspire will resolve it to the gRPC endpoint. The protocol is inferred from the URI scheme.

---

### 3.7 Hands-on: Enhance ServiceDefaults with Custom Metrics and Readiness Health Check

Now let’s put everything together by extending the ServiceDefaults project to support:

- Custom metrics registration.
- A readiness health check that depends on configuration.

#### Step 1: Modify ServiceDefaults to accept additional metrics

Instead of hard‑coding the metrics instruments, we can allow the calling service to pass a delegate to configure metrics. This is a common pattern. Edit `Extensions.cs` in the ServiceDefaults project:

Add a new overload of `AddServiceDefaults` that accepts an `Action<MeterProviderBuilder>`? Actually, the metrics builder is inside `WithMetrics`, so we need to expose that. We can change the `ConfigureOpenTelemetry` method to accept an optional parameter.

But to keep it simple, we’ll create a new method `AddCustomMetrics` that can be called after `AddServiceDefaults`. Modify `ConfigureOpenTelemetry` to be public and accept an optional delegate:

```csharp
public static IHostApplicationBuilder ConfigureOpenTelemetry(
    this IHostApplicationBuilder builder,
    Action<MeterProviderBuilder>? configureMetrics = null)
{
    builder.Logging.AddOpenTelemetry(logging =>
    {
        logging.IncludeFormattedMessage = true;
        logging.IncludeScopes = true;
    });

    builder.Services.AddOpenTelemetry()
        .WithMetrics(metrics =>
        {
            metrics.AddAspNetCoreInstrumentation()
                   .AddHttpClientInstrumentation()
                   .AddRuntimeInstrumentation();
            configureMetrics?.Invoke(metrics);
        })
        .WithTracing(tracing =>
        {
            tracing.AddAspNetCoreInstrumentation()
                   .AddHttpClientInstrumentation()
                   .AddEntityFrameworkCoreInstrumentation();
        });

    builder.Services.AddOpenTelemetryExporters();

    return builder;
}
```

Then update `AddServiceDefaults` to call `ConfigureOpenTelemetry` without a delegate, preserving backward compatibility.

#### Step 2: Use the new overload in your service

In `ApiService/Program.cs`, after `AddServiceDefaults`, call `ConfigureOpenTelemetry` again with a delegate to add your custom meter:

```csharp
builder.AddServiceDefaults();

builder.ConfigureOpenTelemetry(metrics =>
{
    metrics.AddMeter("MyApp.Orders");
});
```

This adds your custom meter while retaining all default instrumentations.

#### Step 3: Add a readiness health check based on configuration

Sometimes you want a health check that depends on a feature flag or configuration setting. Let’s create a health check that fails if a certain configuration key is missing.

In `ApiService/HealthChecks`, create `ConfigHealthCheck.cs`:

```csharp
using Microsoft.Extensions.Diagnostics.HealthChecks;
using Microsoft.Extensions.Options;

namespace MyAspireApp.ApiService.HealthChecks;

public class ConfigHealthCheck : IHealthCheck
{
    private readonly IConfiguration _configuration;

    public ConfigHealthCheck(IConfiguration configuration)
    {
        _configuration = configuration;
    }

    public Task<HealthCheckResult> CheckHealthAsync(
        HealthCheckContext context,
        CancellationToken cancellationToken = default)
    {
        var requiredSetting = _configuration["RequiredSetting"];
        if (string.IsNullOrEmpty(requiredSetting))
        {
            return Task.FromResult(HealthCheckResult.Unhealthy("RequiredSetting is missing"));
        }
        return Task.FromResult(HealthCheckResult.Healthy("RequiredSetting is present"));
    }
}
```

Register it in `Program.cs` with the “ready” tag:

```csharp
builder.Services.AddHealthChecks()
    .AddCheck<DatabaseHealthCheck>("database", tags: ["ready"])
    .AddCheck<ConfigHealthCheck>("config", tags: ["ready"]);
```

Now map readiness endpoint:

```csharp
app.MapHealthChecks("/health/ready", new HealthCheckOptions
{
    Predicate = check => check.Tags.Contains("ready")
});
```

#### Step 4: Run and observe

Run the AppHost. In the dashboard, you won’t see the new health checks unless you map the endpoint. But you can manually hit `/health/ready` on the API service to see the result. If you haven’t set `RequiredSetting` in configuration, the check will fail. To set it, add to `appsettings.json` or set an environment variable.

---

### 3.8 Summary

In this chapter, we’ve taken a deep dive into each aspect of the ServiceDefaults project:

- **OpenTelemetry**: We explored traces, metrics, logs, and how to add custom metrics.
- **Health checks**: We learned about liveness vs. readiness and created advanced checks.
- **Resilience**: We saw the default Polly policies and how to customize them.
- **Service discovery**: We understood the environment variable mechanism and how to use it.

You now have the knowledge to not only use the defaults but also tailor them to your application’s needs. In the next chapter, we’ll start adding real infrastructure—like a PostgreSQL database—and see how Aspire components make integration trivial.

---

**Exercises**

1. Add a custom metric to count the number of times a specific endpoint is called. Verify that the metric appears in the Aspire Dashboard (you may need to enable metrics exploration in the dashboard).
2. Create a health check that verifies connectivity to an external API (e.g., a public weather API). Use `IHttpClientFactory` to make the call.
3. Customize the resilience handler for the `WeatherApiClient` to have a longer timeout and more retries, while leaving other clients with the defaults.

In the next chapter, we’ll integrate a PostgreSQL database using the `Aspire.Npgsql` component and see how Aspire manages connection strings and health checks automatically.